In [ ]:
import sys
!{sys.executable} -m pip install pillow
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install tensorflow
!{sys.executable} -m pip install keras
!{sys.executable} -m pip install -q tqdm
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install pydot
!{sys.executable} -m pip install pydot-ng
!{sys.executable} -m pip install graphviz
!{sys.executable} -m pip install pydotplus
!{sys.executable} -m pip install knockknock
!{sys.executable} -m pip install keyring
!{sys.executable} -m pip install nltk

In [ ]:
import tensorflow as tf

# You'll generate plots of attention in order to see which parts of an image
# our model focuses on during captioning
import matplotlib.pyplot as plt

# Scikit-learn includes many helpful utilities
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import re
import numpy as np
import os
import time
import json
from glob import glob
from PIL import Image
import pickle
from tqdm import tqdm
import pandas as pd
from knockknock import telegram_sender

# Currently the ‘memory growth’ option should be the same for all GPUs.
# You should set the ‘memory growth’ option before initializing GPUs.

import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
  except RuntimeError as e:
    print(e)

In [ ]:
# Download image files
image_folder_flickr = '/flickr30k_images/'
if not os.path.exists(os.path.abspath('.') + image_folder_flickr):
  image_zip = tf.keras.utils.get_file('dataset.zip',
                                      cache_subdir=os.path.abspath('.'),
                                      origin = 'https://casacalarota.onthewifi.com/wp-content/uploads/2020/10/archive.zip',
                                      extract = True)
  PATH_flickr = os.path.dirname(image_zip) + image_folder_flickr
  os.remove(image_zip)
else:
  PATH_flickr = os.path.abspath('.') + image_folder_flickr

In [ ]:
# extract result.csv
result = pd.read_csv(PATH_flickr+'results.csv', sep="|", error_bad_lines=False)
result = result.replace({np.nan: None})
result.columns
annotations_flickr = result.groupby('image_name')[' comment'].apply(list).to_dict()
annotations_flickr

In [ ]:
# Download caption annotation files
annotation_folder_coco = '/annotations/'
if not os.path.exists(os.path.abspath('.') + annotation_folder_coco):
  annotation_zip = tf.keras.utils.get_file('captions.zip',
                                          cache_subdir=os.path.abspath('.'),
                                          origin = 'http://images.cocodataset.org/annotations/annotations_trainval2014.zip',
                                          extract = True)
  # annotation_file = os.path.dirname(annotation_zip)+'/annotations/captions_train2014.json'
  os.remove(annotation_zip)
annotation_file_coco = os.path.abspath('.')+'/annotations/captions_train2014.json'

# Download image files
image_folder_coco = '/train2014/'
if not os.path.exists(os.path.abspath('.') + image_folder_coco):
  image_zip = tf.keras.utils.get_file('train2014.zip',
                                      cache_subdir=os.path.abspath('.'),
                                      origin = 'http://images.cocodataset.org/zips/train2014.zip',
                                      extract = True)
  PATH_coco = os.path.dirname(image_zip) + image_folder_coco
  os.remove(image_zip)
else:
  PATH_coco = os.path.abspath('.') + image_folder_coco

In [ ]:

# Read the json file
with open(annotation_file_coco, 'r') as f:
    annotations_coco = json.load(f)

# Store captions and image names in vectors
all_captions = []
all_img_name_vector = []

for annot in annotations_coco['annotations']:
    caption = '<start> ' + annot['caption'] + ' <end>'
    image_id = annot['image_id']
    full_coco_image_path = PATH_coco + 'COCO_train2014_' + '%012d.jpg' % (image_id)

    all_img_name_vector.append(full_coco_image_path)
    all_captions.append(caption)
    

# Read the json file
# with open(annotation_file, 'r') as f:
#    annotations = json.load(f)



for filename in annotations_flickr:
    caption_list = annotations_flickr[filename]
    for caption in caption_list:
      if isinstance(caption,str):
        caption_parsed = '<start>' + caption + ' <end>'
        all_captions.append(caption_parsed)
        image_id = filename
        full_coco_image_path = PATH_flickr + 'flickr30k_images/' + image_id

        all_img_name_vector.append(full_coco_image_path)
    

# Shuffle captions and image_names together
# Set a random state
train_captions, img_name_vector = shuffle(all_captions,
                                          all_img_name_vector,
                                          random_state=1)
    

# Shuffle captions and image_names together
# Set a random state
train_captions, img_name_vector = shuffle(all_captions,
                                          all_img_name_vector,
                                          random_state=1)
threshold = 400000
# seleziono solo 30k captions (circa 6k immagini)
train_captions = train_captions[:threshold]
img_name_vector = img_name_vector[:threshold]
img_name_vector

In [ ]:
# Choose the top 5000 words from the vocabulary
# preprocessing standard, limita le parole alle 10k più fequenti,
# sostituisce le parole non conosciute (quindi dalla 10k+1 in poi) con
# il singoletto "<unk>" e sassa tutti i caratteri speciali vari
top_k = 10000
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=top_k,
                                                  oov_token="<unk>",
                                                  filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')

# fitta il tokenizer che abbiamo creato nella riga precedente con il set
# di caption
tokenizer.fit_on_texts(train_captions)

# sostituisce le parole con gli int token corrispondenti
train_seqs = tokenizer.texts_to_sequences(train_captions)

In [ ]:
# forza l'indice 0 del tokenizer ad essere il singoletto "<pad>"
tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'

In [ ]:
# Create the tokenized vectors
# fa di nuovo sta cosa (per il pad suppongo)
train_seqs = tokenizer.texts_to_sequences(train_captions)

In [ ]:
# Pad each vector to the max_length of the captions
# If you do not provide a max_length value, pad_sequences calculates it automatically

# padda tutto senza lunghezza fissata (prende la più lunga). paddando mette degli 0
# ma gli zeri abbiamo visto che sono <pad>
cap_vector = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, padding='post')


In [ ]:
# Create training and validation sets using an 80-20 split
# classico split, ritorna 4 valori (due di test e due di train ovviamente)
img_name_train, img_name_val, cap_train, cap_val = train_test_split(img_name_vector,
                                                                    cap_vector,
                                                                    test_size=0.2,
                                                                    random_state=0)


In [ ]:
len(img_name_train), len(cap_train), len(img_name_val), len(cap_val)

In [ ]:
# Feel free to change these parameters according to your system's configuration

embedding_dim = 256
units = 512
top_k = 10000
vocab_size = top_k + 1
# Shape of the vector extracted from InceptionV3 is (64, 2048)
# These two variables represent that vector shape
features_shape = 2048
attention_features_shape = 64


max_length=500

In [ ]:
foldername = "MS_COCO_FLICKR30"
from pathlib import Path
curDir = os.getcwd()
Path(curDir + f"/model/{foldername}/").mkdir(parents=True, exist_ok=True)
Path(curDir + f"/tokenizer/{foldername}/").mkdir(parents=True, exist_ok=True)
tokenizer_file_name = f"tokenizer/{foldername}/tokenizer.pickle"
encoder_file_name = f"model/{foldername}/encoder"
decoder_file_name = f"model/{foldername}/decoder"


In [ ]:
# load tokenizer
from pickle import load
tokenizer = load(open(tokenizer_file_name,"rb"))

In [ ]:
# subclassare Model è uno dei due modi di tf per creare un Model
# usando questo metodo è necessario definire il layer in __init__
# e implementare un metodo "call" che definisca il model's forward pass
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__() #standard
    
    #units è globale ed è 512. Questo è un normale layer Dense
    #cioè un layer fully connected. L'activation è di default è linear (quindi non c'è)
    #ci possiamo giocare volendo per vedere se migliora il tutto
    self.W1 = tf.keras.layers.Dense(units)
    
    #units è globale ed è 512. Questo è un normale layer Dense
    #cioè un layer fully connected. L'activation è di default è linear (quindi non c'è)
    #ci possiamo giocare volendo per vedere se migliora il tutto
    self.W2 = tf.keras.layers.Dense(units)
    
    #classico layer Dense di output, in modo che ci sia un solo output
    self.V = tf.keras.layers.Dense(1)

  def call(self, features, hidden):
    # features(CNN_encoder output) shape == (batch_size, 64, embedding_dim)

    # hidden shape == (batch_size, hidden_size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden_size)
    hidden_with_time_axis = tf.expand_dims(hidden, 1)

    # score shape == (batch_size, 64, hidden_size)
    score = tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis))

    # attention_weights shape == (batch_size, 64, 1)
    # you get 1 at the last axis because you are applying score to self.V
    attention_weights = tf.nn.softmax(self.V(score), axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * features
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [ ]:
class CNN_Encoder(tf.keras.Model):
    # Since you have already extracted the features and dumped it using pickle
    # This encoder passes those features through a Fully connected layer
    def __init__(self, embedding_dim):
        super(CNN_Encoder, self).__init__()
        # shape after fc == (batch_size, 64, embedding_dim)
        self.fc = tf.keras.layers.Dense(embedding_dim)

    def call(self, x):
        x = self.fc(x)
        x = tf.nn.relu(x)
        return x

In [ ]:
class RNN_Decoder(tf.keras.Model):
  def __init__(self, embedding_dim, units, vocab_size):
    super(RNN_Decoder, self).__init__()
    self.units = units

    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc1 = tf.keras.layers.Dense(self.units)
    self.fc2 = tf.keras.layers.Dense(vocab_size)

    self.attention = BahdanauAttention(self.units)

  def call(self, x, features, hidden):
    # defining attention as a separate model
    context_vector, attention_weights = self.attention(features, hidden)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # shape == (batch_size, max_length, hidden_size)
    x = self.fc1(output)

    # x shape == (batch_size * max_length, hidden_size)
    x = tf.reshape(x, (-1, x.shape[2]))

    # output shape == (batch_size * max_length, vocab)
    x = self.fc2(x)

    return x, state, attention_weights

  def reset_state(self, batch_size):
    return tf.zeros((batch_size, self.units))

In [ ]:
# Load weights
encoder = CNN_Encoder(embedding_dim)
decoder = RNN_Decoder(embedding_dim, units, vocab_size)
encoder.load_weights(encoder_file_name)
decoder.load_weights(decoder_file_name)

In [ ]:
# inizializzo un'istanza del modelo InceptionV3 trainato su imagenet (classificazione immagini)
image_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')
     
# l'input del mio modello è uguale all'input di InceptionV 3                                            
new_input = image_model.input
# l'output del mio modelllo è uguale all'ultimo layer di InceptionV3, una convnet con attenzione
hidden_layer=image_model.layers[-1].output

# creo il mio modello
image_features_extract_model = tf.keras.Model(new_input, hidden_layer)

In [ ]:
#funzione per il preprocessing delle immagini in modo che siano coerenti con
#l'input di InceptionV3
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.adjust_jpeg_quality(img, jpeg_quality=10)
    img = tf.image.resize(img, (299, 299))
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img, image_path

#funzione per mostrare il preprocessing fatto su un'immagine e quella originale
def visualize(im_path, imAgmented, operation):
    temp_image = np.array(Image.open(im_path))
    fig = plt.figure()
    plt.subplot(1,2,1)
    plt.title('Original image')
    plt.imshow(temp_image)
    plt.subplot(1,2,2)
    plt.title(operation)
    plt.imshow(imAgmented)

In [ ]:
def evaluate(image):
    attention_plot = np.zeros((max_length, attention_features_shape))

    hidden = decoder.reset_state(batch_size=1)

    temp_input = tf.expand_dims(load_image(image)[0], 0)
    img_tensor_val = image_features_extract_model(temp_input)
    img_tensor_val = tf.reshape(img_tensor_val, (img_tensor_val.shape[0], -1, img_tensor_val.shape[3]))

    features = encoder(img_tensor_val)

    dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)
    result = []

    for i in range(max_length):
        predictions, hidden, attention_weights = decoder(dec_input, features, hidden)

        attention_plot[i] = tf.reshape(attention_weights, (-1, )).numpy()

        predicted_id = tf.random.categorical(predictions, 1)[0][0].numpy()
        result.append(tokenizer.index_word[predicted_id])

        if tokenizer.index_word[predicted_id] == '<end>':
            return result, attention_plot

        dec_input = tf.expand_dims([predicted_id], 0)

    attention_plot = attention_plot[:len(result), :]
    return result, attention_plot


In [ ]:
def plot_attention(image, result, attention_plot):
    temp_image = np.array(Image.open(image))

    fig = plt.figure(figsize=(10, 10))

    len_result = len(result)
    for l in range(len_result):
        temp_att = np.resize(attention_plot[l], (8, 8))
        ax = fig.add_subplot(len_result//2, len_result//2, l+1)
        ax.set_title(result[l])
        img = ax.imshow(temp_image)
        ax.imshow(temp_att, cmap='gray', alpha=0.6, extent=img.get_extent())

    plt.tight_layout()
    plt.show()


In [ ]:
from nltk import ngrams

def get_Xigram(caption, n):
    return list(ngrams(caption.split(), n))


def score_Xigram(real_caption, predicted_caption, ngram=1):
    real_Xigrams = get_Xigram(real_caption, ngram)
    predicted_Xigrams = get_Xigram(predicted_caption, ngram)
    total_number_hypothesis = len(real_Xigrams)
    return sum([predicted_Xigrams.count(Xigram) for Xigram in real_Xigrams]) / total_number_hypothesis

In [ ]:
CHAT_ID: int = ""
@telegram_sender(token="", chat_id=CHAT_ID)
def total_score():
    sum_score_1 = 0
    sum_score_2 = 0
    sum_score_3 = 0
    for index, image in enumerate(img_name_val):
        # print(image)
        real_caption =  ' '.join([tokenizer.index_word[i] for i in cap_val[index] if i not in [0]])
        result, attention_plot = evaluate(image)
        predicted_caption = '<start> ' + ' '.join(result)
        # print(predicted_caption)
        # print(real_caption)
        # print(score_single_image)
        sum_score_1 += score_Xigram(real_caption, predicted_caption, ngram=1)
        sum_score_2 += score_Xigram(real_caption, predicted_caption, ngram=2)
        sum_score_3 += score_Xigram(real_caption, predicted_caption, ngram=3)
        if index % 100 == 0:
            print("Img: ", image)
            print ('Real Caption:', real_caption)
            print ('Prediction Caption:', predicted_caption)
            plot_attention(image, result, attention_plot)
            perc_completed = (index + 1) / len(img_name_val) * 100
            print("Percent test_set completed={perc_completed}".format(perc_completed=perc_completed))
    len_img_val = len(img_name_val)
    return sum_score_1 / len_img_val, sum_score_2 / len_img_val, sum_score_3 / len_img_val

unigram, bigram, trigram = total_score()
print("UNIGRAM TOTAL SCORE: ", unigram)
print("BIGRAM TOTAL SCORE: ", bigram)
print("TRIGRAM TOTAL SCORE: ", trigram)

In [ ]:
# captions on the validation set
rid = np.random.randint(0, len(img_name_val))
print(rid)
image = img_name_val[rid]
print(image)
real_caption = ' '.join([tokenizer.index_word[i] for i in cap_val[rid] if i not in [0]])
result, attention_plot = evaluate(image)

print ('Real Caption:', real_caption)
print ('Prediction Caption:', ' '.join(result))
plot_attention(image, result, attention_plot)

Image.open(image)

In [ ]:
import requests
# TESTING PURPOSES
from pathlib import Path
curDir = os.getcwd()
Path(curDir + "/test/").mkdir(parents=True, exist_ok=True)
image_path = curDir+'/test/image.jpg'
image_url = "https://hips.hearstapps.com/hmg-prod.s3.amazonaws.com/images/instagram-captions-for-couples-1578955100.jpg"
with open(image_path, 'wb') as f:
    f.write(requests.get(image_url).content)



result, attention_plot = evaluate(image_path)
print ('Prediction Caption:', ' '.join(result))
plot_attention(image_path, result, attention_plot)
# opening the image
Image.open(image_path)